<a href="https://colab.research.google.com/github/Tiabet/DACON-code/blob/master/%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [43]:
# 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [24]:
train_df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


1. 대출 목적 비슷한 것들은 묶기
2. 근로기간 언노운 처리하고 1이하랑 1 묶기
3. 주택소유상태 ANY 처리
4. 연체계좌수, 총연체금액 드랍

In [55]:
train_df.drop(['연체계좌수', '총연체금액'], axis=1, inplace = True)

In [56]:
train_df[train_df['주택소유상태']=='ANY']['주택소유상태'] = 'MORTGAGE'

<ipython-input-56-490cd61a9ff0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[train_df['주택소유상태']=='ANY']['주택소유상태'] = 'MORTGAGE'


In [57]:
train_df['주택소유상태'].value_counts()

MORTGAGE    47934
RENT        37705
OWN         10654
ANY             1
Name: 주택소유상태, dtype: int64

In [44]:
train_df.loc[train_df['근로기간']=='3','근로기간']='3 years'
train_df.loc[train_df['근로기간']=='1 year','근로기간']='1 years'
train_df.loc[train_df['근로기간']=='<1 year','근로기간']='< 1 year'
train_df.loc[train_df['근로기간']=='10+years','근로기간']='10+ years'
train_df.value_counts('근로기간')

근로기간
10+ years    32481
2 years       8450
< 1 year      8144
3 years       7670
1 years       6305
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
dtype: int64

In [45]:
values_map=dict({'6 years':6,
 '10+ years':10,
 '5 years':3,
 '8 years':7,
 '9 years':7,
 '2 years':3,
 '1 years':1,
 '3 years':3,
 '7 years':7,
 '4 years':3,
 '< 1 year':1})

In [46]:
train_df['근로기간'] = train_df['근로기간'].replace(values_map)

In [47]:
train_df.loc[train_df['근로기간']=='Unknown','근로기간']= np.random.randint(1,10)

In [48]:
train_df.loc[train_df['근로기간']==6,'근로기간']= np.random.randint(1,3)

In [49]:
train_df['근로기간'].value_counts()

10    32481
3     27373
1     18323
7     18117
Name: 근로기간, dtype: int64

In [51]:
train_df['대출목적'].value_counts()

부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4725
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: 대출목적, dtype: int64

In [53]:
train_df.loc[~train_df['대출목적'].isin(['부채 통합', '신용 카드', '기타']), '대출목적'] = '기타'